In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("spark-notebook") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider") \
    .getOrCreate()

In [4]:
spark

In [5]:
spark.sql("SELECT 'Hello, World!'").show()

+-------------+
|Hello, World!|
+-------------+
|Hello, World!|
+-------------+



In [6]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("s3a://openaq-data-archive/records/csv.gz/locationid=2178/year=2022/month=05/location-2178-20220503.csv.gz")

df.show(10)

25/11/29 18:55:32 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+-----------+----------+--------------+-------------------+-------+-----------+---------+-----+-----+
|location_id|sensors_id|      location|           datetime|    lat|        lon|parameter|units|value|
+-----------+----------+--------------+-------------------+-------+-----------+---------+-----+-----+
|       2178|      3919|Del Norte-2178|2022-05-03 07:00:00|35.1353|-106.584702|     pm10|µg/m³| 31.0|
|       2178|      3919|Del Norte-2178|2022-05-03 08:00:00|35.1353|-106.584702|     pm10|µg/m³| 22.0|
|       2178|      3919|Del Norte-2178|2022-05-03 09:00:00|35.1353|-106.584702|     pm10|µg/m³| 29.0|
|       2178|      3919|Del Norte-2178|2022-05-03 10:00:00|35.1353|-106.584702|     pm10|µg/m³| 30.0|
|       2178|      3919|Del Norte-2178|2022-05-03 11:00:00|35.1353|-106.584702|     pm10|µg/m³| 31.0|
|       2178|      3919|Del Norte-2178|2022-05-03 12:00:00|35.1353|-106.584702|     pm10|µg/m³| 44.0|
|       2178|      3919|Del Norte-2178|2022-05-03 13:00:00|35.1353|-106.584702|   

In [7]:
df.write.format("jdbc") \
    .option("url", "jdbc:postgresql://db:5432/postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", "spark_data.air_quality") \
    .option("user", "admin") \
    .option("password", "password") \
    .mode("overwrite") \
    .save()